# Tutorial Showing the Interoperability of Full-Rank-Retrievers and Re-Rankers Submitted to TIRA

This notebook shows how post-hoc experiments of the IR Experiment Platform can be conducted.

To start the notebook, please clone the archived shared task repository:

```
git@github.com:tira-io/ir-experiment-platform-benchmarks.git
```

Inside the cloned repository, you can start the Jupyter notebook which automatically installs a minimal virtual environment using:
```
make jupyterlab
```

The notebook covers how to combine full-rank appraoches submitted to TIRA with re-rank approaches submitted to TIRA in reproducibility/replicability experiments on the same or new data.

For each of the softwares submitted to TIRA, the `tira` integration to PyTerrier loads the Docker Image submitted to TIRA to execute it in PyTerrier pipelines (i.e., a first execution could take sligthly longer).


In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', 0)

from tira.local_client import Client
tira = Client()

import pyterrier as pt
if not pt.started():
    pt.init()


### Initialize A Full-Rank Retriever

We create a pyterrier retriever called `submitted_baseline` that is an approach submitted to a shared task in TIRA.
The approach is identified by the name `ir-benchmarks/tira-ir-starter/BM25 (tira-ir-starter-pyterrier)`, i.e., a software `BM25 (tira-ir-starter-pyterrier)` submitted to `ir-benchmarks` by the team `tira-ir-starter` (that hosts baselines).
This software consists of two stages: First, a first software component builds an PyTerrier Index, and the second software does the actual retrieval with BM25.

With this API, any full-rank approach submitted in TIRA can be executed and re-executed, e.g., on new data.

We can run the retriever on any dataset integrated in `ir_dataset`.
Here, we use `vaswani` to show the overall functionality with a fast example.

In [4]:
submitted_baseline = tira.pt.retriever(
    'ir-benchmarks/tira-ir-starter/BM25 (tira-ir-starter-pyterrier)',
    dataset='vaswani',
)


Next, we can make the actual retrieval, here on two topics to keep the result set size small.

In [5]:
topics = pd.DataFrame([
    {'qid': 1, 'query': 'MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES'},
    {'qid': 2, 'query': 'MATHEMATICAL ANALYSIS AND DESIGN DETAILS OF WAVEGUIDE FED MICROWAVE RADIATIONS'},
])

submitted_baseline(topics)

,qid,query,q0,docno,rank,score,system
0,1,MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES,Q0,8172,1,24.566031,pyterrier.default_pipelines.wmodel_batch_retrieve
1,1,MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES,Q0,9881,2,22.110514,pyterrier.default_pipelines.wmodel_batch_retrieve
2,1,MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES,Q0,5502,3,21.717148,pyterrier.default_pipelines.wmodel_batch_retrieve
3,1,MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES,Q0,1502,4,19.478355,pyterrier.default_pipelines.wmodel_batch_retrieve
4,1,MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES,Q0,9859,5,18.626342,pyterrier.default_pipelines.wmodel_batch_retrieve
...,...,...,...,...,...,...,...
1995,2,MATHEMATICAL ANALYSIS AND DESIGN DETAILS OF WAVEGUIDE FED MICROWAVE RADIATIONS,Q0,4833,996,5.161525,pyterrier.default_pipelines.wmodel_batch_retrieve
1996,2,MATHEMATICAL ANALYSIS AND DESIGN DETAILS OF WAVEGUIDE FED MICROWAVE RADIATIONS,Q0,3529,997,5.161525,pyterrier.default_pipelines.wmodel_batch_retrieve
1997,2,MATHEMATICAL ANALYSIS AND DESIGN DETAILS OF WAVEGUIDE FED MICROWAVE RADIATIONS,Q0,271,998,5.161525,pyterrier.default_pipelines.wmodel_batch_retrieve
1998,2,MATHEMATICAL ANALYSIS AND DESIGN DETAILS OF WAVEGUIDE FED MICROWAVE RADIATIONS,Q0,2429,999,5.161525,pyterrier.default_pipelines.wmodel_batch_retrieve


Next, we create an `advanced_baseline` that re-ranks the top 10 results of the `submitted_baseline` that was submitted to TIRA with another re-ranker that was also submitted to TIRA, i.e., with `ir-benchmarks/tira-ir-starter/SBERT multi-qa-MiniLM-L6-dot-v1 (tira-ir-starter-beir)`.

All full-rank approaches submitted in TIRA can be the first stage for any second-stage re-ranker (or longer chains).
This is ensured by the ir_datasets integration that ensures that all softwares are interoperable.
In this case, the ir_datasets integration automatically runs in between.



In [6]:
advanced_baseline = submitted_baseline %10 >> tira.pt.reranker(
    'ir-benchmarks/tira-ir-starter/SBERT multi-qa-MiniLM-L6-dot-v1 (tira-ir-starter-beir)',
    irds_id='vaswani'
)

advanced_baseline(topics)

Task: Re-Rank -> create files: 
 rerank.jsonl 
 qrels.txt 
 at /output/
Get Documents
Produce rerank data.
Write rerank data.
Done rerank data was written.


Get Docs: 100%|██████████| 18/18 [00:00<00:00, 3821.69it/s]
Produce Rerank File.: 18it [00:00, 12110.60it/s]


,qid,query,docno,q0,rank,score,system
0,1,MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES,8172,0,7,37.618500,multi-qa-MiniLM-L6-dot-v1-dot
1,1,MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES,9881,0,3,39.492393,multi-qa-MiniLM-L6-dot-v1-dot
2,1,MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES,5502,0,1,45.553276,multi-qa-MiniLM-L6-dot-v1-dot
3,1,MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES,1502,0,2,45.179565,multi-qa-MiniLM-L6-dot-v1-dot
4,1,MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES,9859,0,9,36.344490,multi-qa-MiniLM-L6-dot-v1-dot
5,1,MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES,4871,0,5,38.651424,multi-qa-MiniLM-L6-dot-v1-dot
6,1,MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES,4817,0,4,38.860371,multi-qa-MiniLM-L6-dot-v1-dot
7,1,MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES,8276,0,6,37.756004,multi-qa-MiniLM-L6-dot-v1-dot
8,1,MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES,7234,0,8,37.546501,multi-qa-MiniLM-L6-dot-v1-dot
9,2,MATHEMATICAL ANALYSIS AND DESIGN DETAILS OF WAVEGUIDE FED MICROWAVE RADIATIONS,2850,0,4,38.820084,multi-qa-MiniLM-L6-dot-v1-dot
